# Final Project Overview
Your boss asked you to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, you need to transform the data and store it in USD, GBP, EUR, and INR per the exchange rate information made available to you as a CSV file. You should save the processed information table locally in a CSV format and as a database table. Managers from different countries will query the database table to extract the list and note the market capitalization value in their own currency.

## Direction
1. Write a function to extract the tabular information from the given URL under the heading By Market Capitalization, and save it to a data frame.
2. Write a function to transform the data frame by adding columns for Market Capitalization in GBP, EUR, and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.
3. Write a function to load the transformed data frame to an output CSV file.
4. Write a function to load the transformed data frame to an SQL database server as a table.
5. Write a function to run queries on the database table.
6. Run the following queries on the database table:
   - Extract the information for the London office, that is Name and MC_GBP_Billion
   - Extract the information for the Berlin office, that is Name and MC_EUR_Billion
   - Extract the information for New Delhi office, that is Name and MC_INR_Billion
8. Write a function to log the progress of the code.
9. While executing the data initialization commands and function calls, maintain appropriate log entries.

importing packages/libraries

In [107]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import sqlite3

Creating a log function

In [99]:
def log_info(data):
    file = 'code_log.txt'
    timeStamp_format = '%Y-%m-%d %H:%M:%S'
    timeStamp = datetime.now().strftime(timestamp_format)
    with open(file, 'a+') as f:
        f.write(f'{timeStamp} - {data}\n')

Accessing URL data

In [135]:
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
output_CSV_path = './Largest_banks_data.csv'
db_name = 'Banks.db'
db_table_name = 'Largest_banks'
log_file = 'code_log.txt'
exchange_rates = 'exchange_rate.csv'

In [144]:
def extract(url, table_attribs):
    response = requests.get(url)
    log_info(f'URL Fetched: status code is {response.status_code}')
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find_all('table')[0]
        rows = table.find_all('tr')
        rows = rows[1:]
        data = {}
        # data['rank'] = []
        data[table_attribs[0]] = []
        data[table_attribs[1]] = []
        for row in rows:
            # rank = int(row.find_all('td')[0].text.strip())
            bank_name = row.find_all('td')[1].find_all('a')[-1].text
            market_cap = float(row.find_all('td')[2].text.strip())
            # data['rank'].append(rank)
            data[table_attribs[0]].append(bank_name)
            data[table_attribs[1]].append(market_cap)
        df = pd.DataFrame(data)
        log_info('Extraction: data extracted and loaded into a DataFrame')
        return df

In [143]:
df = extract(url,['bank_name', 'market_cap(USD_billion)'])


int

In [140]:
def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    ex_df = pd.read_csv(exchange_rates, index_col='Currency') # Converting currency into a DF
    df['market_cap(EUR_billion)'] = df['market_cap(USD_billion)']*ex_df.loc['EUR','Rate']
    df['market_cap(GBP_billion)'] = df['market_cap(USD_billion)']*ex_df.loc['GBP','Rate']
    df['market_cap(INR_billion)'] = df['market_cap(USD_billion)']*ex_df.loc['INR','Rate']
    log_info('Transformation: New currency added for EUR, GBP and INR')
    return df

In [141]:
new_df = transform(df, exchange_rates)
new_df

,bank_name,market_cap(USD_billion),market_cap(EUR_billion),market_cap(GBP_billion),market_cap(INR_billion)
0,JPMorgan Chase,432.92,402.6156,346.336,35910.7140
1,Bank of America,231.52,215.3136,185.216,19204.5840
2,Industrial and Commercial Bank of China,194.56,180.9408,155.648,16138.7520
3,Agricultural Bank of China,160.68,149.4324,128.544,13328.4060
4,HDFC Bank,157.91,146.8563,126.328,13098.6345
5,Wells Fargo,155.87,144.9591,124.696,12929.4165
6,HSBC Holdings PLC,148.90,138.4770,119.120,12351.2550
7,Morgan Stanley,140.83,130.9719,112.664,11681.8485
8,China Construction Bank,139.82,130.0326,111.856,11598.0690
9,Bank of China,136.81,127.2333,109.448,11348.3895
